# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [521]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=100" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 105124
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3600
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=100 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=100"
DEBUG:cachecontrol.controller:Caching b/c of expires header


In [522]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [523]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,11 cities ride 3/3 race,02-20 08:00,/events/64068,[race],4,93.51 km,91 m,91 m,11 City's Ride 3_3,/courses/118499,False
1,Killer-manjaro Base Camp,02-20 08:30,/events/64249,[race],52,48.72 km,2.36 km,0 m,Kilimanjaro Base Camp,/courses/124659,False
2,HCC Group Ride,02-20 09:00,/events/68050,[groupride],5,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False
3,Virtuslo Medio Fondo,02-20 09:00,/events/49798,[race],61,98.00 km,1.64 km,1.57 km,TopFit Flanders,/courses/104392,True
4,Tour de Waffle,02-20 09:00,/events/59386,[race],25,25.16 km,614 m,614 m,Paterberg,/courses/88,False
...,...,...,...,...,...,...,...,...,...,...,...
95,Breakfast Club,02-23 07:00,/events/59689,[groupride],1,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
96,Gran Premio Pienza,02-23 09:00,/events/59255,[race],2,24.63 km,706 m,706 m,Pienza,/courses/64,False
97,Chat Laps,02-23 10:00,/events/59750,[groupride],2,24.63 km,706 m,706 m,Pienza,/courses/64,False
98,Breakfast Club,02-23 12:00,/events/59717,[groupride],4,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [524]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [525]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-02-20 08:00:00+00:00,11 cities ride 3/3 race,02-20 08:00,/events/64068,[race],4,93.51 km,91 m,91 m,11 City's Ride 3_3,/courses/118499,False
2021-02-20 08:30:00+00:00,Killer-manjaro Base Camp,02-20 08:30,/events/64249,[race],52,48.72 km,2.36 km,0 m,Kilimanjaro Base Camp,/courses/124659,False
2021-02-20 09:00:00+00:00,HCC Group Ride,02-20 09:00,/events/68050,[groupride],5,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False
2021-02-20 09:00:00+00:00,Virtuslo Medio Fondo,02-20 09:00,/events/49798,[race],61,98.00 km,1.64 km,1.57 km,TopFit Flanders,/courses/104392,True
2021-02-20 09:00:00+00:00,Tour de Waffle,02-20 09:00,/events/59386,[race],25,25.16 km,614 m,614 m,Paterberg,/courses/88,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [526]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-02-20 08:00:00+00:00 11 cities ride 3/3 race ['race']
2021-02-20 08:30:00+00:00 Killer-manjaro Base Camp ['race']
2021-02-20 09:00:00+00:00 HCC Group Ride ['groupride']
2021-02-20 09:00:00+00:00 Virtuslo Medio Fondo ['race']
2021-02-20 09:00:00+00:00 Tour de Waffle ['race']
2021-02-20 09:00:00+00:00 Chat Laps ['groupride']
2021-02-20 09:30:00+00:00 Virtchillee’s Group Ride ['groupride']
2021-02-20 10:00:00+00:00 CTT 25 mile TT ['race', 'itt']
2021-02-20 10:00:00+00:00 Chat Laps ['groupride']
2021-02-20 12:00:00+00:00 Breakfast Club ['groupride']
2021-02-20 12:30:00+00:00 Saturday Ladies Ride ['groupride']
2021-02-20 13:00:00+00:00 Gimbels NY Long ['groupride']
2021-02-20 13:00:00+00:00 Maratona Challenge Part 1 ['groupride']
2021-02-20 14:00:00+00:00 Weekend Warrior ['groupride']
2021-02-20 14:00:00+00:00 NCRA Old Race 7 ['race']
2021-02-20 14:00:00+00:00 J2/9 TT ['race', 'itt']
2021-02-20 15:00:00+00:00 Lou's Saturday Group Ride ['groupride']
2021-02-20 15:00:00+00:00 Chain Gang 

## Write to File

Can use to manually import into Google, other calendars

In [527]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [528]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [532]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [529]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-02-19 09:00:00+00:00 - Race to the Light House ['race'] 2
2021-02-19 15:00:00+00:00 - Napoleon Dolomite ['race'] 1
2021-02-19 15:00:00+00:00 - Chat Laps ['groupride'] 1
2021-02-19 15:00:00+00:00 - Race to the Light House ['race'] 3
2021-02-19 18:00:00+00:00 - Race to the Light House ['race'] 2
2021-02-19 18:00:00+00:00 - SDW Friday Frenzy #2 ['race'] 36
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 1 ['elimination', 'race'] 1
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 2 ['elimination', 'race'] 1
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 3 ['elimination', 'race'] 4
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 4 ['elimination', 'race'] 7
2021-02-20 09:00:00+00:00 - HCC Group Ride ['groupride'] 2
2021-02-20 09:00:00+00:00 - Chat Laps ['groupride'] 2
2021-02-20 09:00:00+00:00 - Tour de Waffle ['race'] 3
2021-02-20 10:00:00+00:00 - CTT 25 mile TT ['race', 'itt'] 5
2021-02-19 22:00:00+00:00 - Race to the Light House ['race'] 1/22.43 km/515 m
2021-02-20 00:00:

In [530]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-20 08:00:00+00:00,11 cities ride 3/3 race,02-20 08:00,/events/64068,[race],4,93.51 km,91 m,91 m,11 City's Ride 3_3,/courses/118499,False,lljn69m8po631mgog4ho00b9hc,2021-02-20 08:00:00+00:00 - 11 cities ride 3/3...,True
2021-02-20 08:30:00+00:00,Killer-manjaro Base Camp,02-20 08:30,/events/64249,[race],52,48.72 km,2.36 km,0 m,Kilimanjaro Base Camp,/courses/124659,False,isub7824rassarm6ilf2b6de30,2021-02-20 08:30:00+00:00 - Killer-manjaro Bas...,True
2021-02-20 09:00:00+00:00,HCC Group Ride,02-20 09:00,/events/68050,[groupride],5,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,vt1lcnivla7k7jhg6spv8pohm0,2021-02-20 09:00:00+00:00 - HCC Group Ride ['g...,True
2021-02-20 09:00:00+00:00,Virtuslo Medio Fondo,02-20 09:00,/events/49798,[race],61,98.00 km,1.64 km,1.57 km,TopFit Flanders,/courses/104392,True,3vi15lart66fm3hituhmjuumec,2021-02-20 09:00:00+00:00 - Virtuslo Medio Fon...,True
2021-02-20 09:00:00+00:00,Tour de Waffle,02-20 09:00,/events/59386,[race],25,25.16 km,614 m,614 m,Paterberg,/courses/88,False,p7e7h2l2kbbq8jl1fd6r4d3mfg,2021-02-20 09:00:00+00:00 - Tour de Waffle ['r...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-21 18:00:00+00:00,Race to the Light House,02-21 18:00,/events/59341,[race],2,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,jchrgcrnv279tlidvkfoht3urg,2021-02-21 18:00:00+00:00 - Race to the Light ...,True
2021-02-21 19:00:00+00:00,Weekend Warrior,02-21 19:00,/events/59852,[groupride],2,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,91lj13d3odpof645icfu18t5ks,2021-02-21 19:00:00+00:00 - Weekend Warrior ['...,True
2021-02-21 19:00:00+00:00,Killer-manjaro Summit,02-21 19:00,/events/67828,[race],7,18.20 km,2.34 km,0 m,Kilimanjaro Summit,/courses/124663,False,qblagc0mff6t5838pdnqhlnrjg,2021-02-21 19:00:00+00:00 - Killer-manjaro Sum...,True


In [531]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-20 13:00:00+00:00,Gimbels NY Long,02-20 13:00,/events/67309,[groupride],19,57.09 km,674 m,747 m,Gimbels NY Long,/courses/123049,False,None,None,False
2021-02-20 14:00:00+00:00,J2/9 TT,02-20 14:00,/events/69016,"[race, itt]",34,40.25 km,161 m,197 m,J2/9 TT,/courses/135978,False,None,None,False
2021-02-20 15:00:00+00:00,Lou's Saturday Group Ride,02-20 15:00,/events/68174,[groupride],19,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,None,None,False
2021-02-20 16:00:00+00:00,Vueltina Asturias Stage 2,02-20 16:00,/events/69316,[race],33,55.44 km,742 m,667 m,Gijon-Caravia (St2),/courses/135286,False,None,None,False
2021-02-20 19:00:00+00:00,Killer-manjaro base camp,02-20 19:00,/events/67825,[race],5,48.72 km,2.36 km,0 m,Kilimanjaro Base Camp,/courses/124659,False,None,None,False
2021-02-21 09:00:00+00:00,GFNS GOLDEN E-FONDO,02-21 09:00,/events/56606,[race],74,42.37 km,1.00 km,564 m,E Fondo Golden V5,/courses/109805,False,None,None,False
2021-02-21 09:00:00+00:00,Anti Social Social Ride,02-21 09:00,/events/59839,[groupride],1,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,None,None,False
2021-02-21 10:00:00+00:00,ACBI ADAMASTOR CHALLENGE,02-21 10:00,/events/67695,[race],25,37.45 km,1.22 km,1.22 km,Adamastor Challenge-,/courses/132649,False,None,None,False
2021-02-21 15:30:00+00:00,OTR Women's Weekender,02-21 15:30,/events/69898,[race],2,38.54 km,489 m,534 m,Saint Johns Ramsey,/courses/137266,False,None,None,False


## Add All Events to Google Calendar

In [533]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

#    if row['cal_id'] == None: # Easier to understand?
#        isNewEvent = True
#    else:
#        isNewEvent = False

#    isNewEvent = (row['cal_id'] == None) # Which is easier to understand?

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    if row['found'] == False:

        evntString = row['detailsUrl'].replace('/events/', '')

        print('+New event: ', index, row['name'], row['tags'], eventString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], eventString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

-Updating event:  2021-02-20 08:00:00+00:00 11 cities ride 3/3 race ['race'] 59685
-Updating event:  2021-02-20 08:30:00+00:00 Killer-manjaro Base Camp ['race'] 59685
-Updating event:  2021-02-20 09:00:00+00:00 HCC Group Ride ['groupride'] 59685
-Updating event:  2021-02-20 09:00:00+00:00 Virtuslo Medio Fondo ['race'] 59685
-Updating event:  2021-02-20 09:00:00+00:00 Tour de Waffle ['race'] 59685
-Updating event:  2021-02-20 09:00:00+00:00 Chat Laps ['groupride'] 59685
-Updating event:  2021-02-20 09:30:00+00:00 Virtchillee’s Group Ride ['groupride'] 59685
-Updating event:  2021-02-20 10:00:00+00:00 CTT 25 mile TT ['race', 'itt'] 59685
-Updating event:  2021-02-20 10:00:00+00:00 Chat Laps ['groupride'] 59685
-Updating event:  2021-02-20 12:00:00+00:00 Breakfast Club ['groupride'] 59685
-Updating event:  2021-02-20 12:30:00+00:00 Saturday Ladies Ride ['groupride'] 59685
+New event:  2021-02-20 13:00:00+00:00 Gimbels NY Long ['groupride'] 59685
ID before add: 67309
ID after add: 67309 Re